In [6]:
import os , re
import glob
import psycopg2
import pandas as pd
from sql_queries import *

In [7]:
def process_data(cur, conn, filepath, func):
    # get all files matching extension from directory
    all_files = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root,'*.json'))
        for f in files :
            all_files.append(os.path.abspath(f))

    # get total number of files found
    num_files = len(all_files)
    print('{} files found in {}'.format(num_files, filepath))

    # iterate over files and process
    for i, datafile in enumerate(all_files, 1):
        func(cur, datafile)
        conn.commit()
        print('{}/{} files processed.'.format(i, num_files))

In [8]:
def process_log_file(cur, filepath):
    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextSong action
    
    df1 = df[df['page']=='NextSong']

    # convert timestamp column to datetime
    
    p = pd.DataFrame()
    p['start_time'] = pd.to_datetime(df["ts"], unit='ms',errors='coerce')
    p['hour'] = p["start_time"].dt.hour
    p['day'] = p["start_time"].dt.day 
    p['week'] = p["start_time"].dt.week
    p['month'] = p["start_time"].dt.month
    p['year'] = p["start_time"].dt.year
    p['weekday'] = p["start_time"].dt.weekday
    
    
    # insert time data records
    time_data = p.values.tolist()
    column_labels = p.columns.tolist()
    time_df = p 

    for i, row in time_df.iterrows():
        cur.execute(time_table_insert, list(row))

    # load user table
    user_df = df[["userId", "firstName", "lastName" ,"gender" , "level"]] 

    # insert user records
    for i, row in user_df.iterrows():
        cur.execute(user_table_insert, list(row))

    # insert songplay records
        


    # insert songplay records
    for index, row in df.iterrows():
        
        # get songid and artistid from song and artist tables
        cur.execute(song_select, (row.song, row.artist, row.length))
        results = cur.fetchone()
        
        if results:
            songid, artistid = results
        else:
            songid, artistid = None, None

        # insert songplay record
        songplay_data = ( row.ts, row.userId, row.level, songid, artistid, row.sessionId, row.location, row.userAgent)
        cur.execute(songplay_table_insert, songplay_data)



In [9]:
def main():
    conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
    cur = conn.cursor()
    cur.execute('truncate table time;')
    cur.execute('truncate table users;')
    process_data(cur, conn, filepath='data/log_data', func=process_log_file)
    conn.close()

In [ ]:

if __name__ == "__main__":
    main()

In [4]:
df = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-02-events.json', lines=True)

df

artist       auth firstName gender  itemInSession  \
0      N.E.R.D. FEATURING MALICE  Logged In    Jayden      M              0   
1                           None  Logged In   Stefany      F              0   
2            Death Cab for Cutie  Logged In   Stefany      F              1   
3               Tracy Gang Pussy  Logged In   Stefany      F              2   
4                        Skillet  Logged In     Kevin      M              0   
5              Dance Gavin Dance  Logged In    Marina      F              0   
6                           None  Logged In     Aiden      M              0   
7                          Dalto  Logged In     Aiden      M              1   
8                     Kanye West  Logged In  Makinley      F              0   
9    Jason Mraz & Colbie Caillat  Logged In     Kevin      M              0   
10                  Liz Callaway  Logged In      Lily      F              0   
11              The Decemberists  Logged In      Lily      F              1   
12                          None  Logged In   Kynnedi      F              0   
13            Christina Aguilera  Logged In      Lily      F              2   
14                   OneRepublic  Logged In      Lily      F              3   
15                   Cat Stevens  Logged In   Kynnedi      F              1   
16                  Brett Dennen  Logged In      Lily      F              4   
17                      Godsmack  Logged In      Lily      F              5   
18                          Anis  Logged In      Lily      F              6   
19                 Tiziano Ferro  Logged In      Lily      F              7   
20                 Kylie Auldist  Logged In      Lily      F              8   
21                         Benga  Logged In      Lily      F              9   
22                        T. Rex  Logged In      Lily      F             10   
23    Mexican Institute of Sound  Logged In      Lily      F             11   
24                    Velour 100  Logged In      Lily      F             12   
25               Maria de Barros  Logged In      Lily      F             13   
26                    Toby Keith  Logged In      Lily      F             14   
27               Marques Houston  Logged In      Lily      F             15   
28                Commit Suicide  Logged In      Lily      F             16   
29              Smile Empty Soul  Logged In      Lily      F             17   
..                           ...        ...       ...    ...            ...   
141    Jack Johnson / Paula Fuga  Logged In    Joseph      M              3   
142                     Tub Ring  Logged In    Joseph      M              4   
143                     No Doubt  Logged In  Mohammad      M              0   
144                   Katy Perry  Logged In  Mohammad      M              1   
145           Neutral Milk Hotel  Logged In    Joseph      M              5   
146                Dwight Yoakam  Logged In       Ava      F              0   
147              Ella Fitzgerald  Logged In    Joseph      M              6   
148                    Tony Rohr  Logged In       Ava      F              1   
149                        Braid  Logged In    Joseph      M              7   
150                     Fourmost  Logged In    Joseph      M              8   
151               Catupecu Machu  Logged In    Joseph      M              9   
152                        Creed  Logged In    Joseph      M             10   
153              Ennio Morricone  Logged In    Joseph      M             11   
154                         None  Logged In    Joseph      M             12   
155             Down To The Bone  Logged In    Sylvie      F              0   
156                 Three Drives  Logged In    Sylvie      F              1   
157                         None  Logged In    Austin      M              0   
158            Sebastien Tellier  Logged In    Sylvie      F              2   
159                Lonnie Gordon  Logged In    Sylvie      F              3 

In [5]:
df = pd.read_json('/home/workspace/data/log_data/2018/11/2018-11-01-events.json', lines=True)
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
for index, row in df.iterrows():

    # get songid and artistid from song and artist tables
    cur.execute(song_select, (row.song, row.artist, row.length))
    results = cur.fetchone()
    
    print(results)
    
    if results:
        songid, artistid = results
    else:
        songid, artistid = None, None

None
None
None
None
None
None
None
None
None
None
None
None
None
None
None


In [3]:
print(results)

('SOIAZJW12AB01853F1', 'AR8ZCNI1187B9A069B')


In [ ]:
    # insert songplay record
    songplay_data = df[["ts", "userId", "level" ,"sessionId" ]] 
    cur.execute(songplay_table_insert, songplay_data)
    conn.commit()
    

In [10]:
conn = psycopg2.connect("host=127.0.0.1 dbname=sparkifydb user=student password=student")
cur = conn.cursor()
cur.execute("insert into artists values('Skillet','AR0IAWL1187B9A96F9','Panama','8.4177','-80.11278')")

In [11]:
cur.execute("insert into songs values('SOMJBYD15Z5D4F8557','Mystic World','AR0IAWL1187B9A96F9','178.02404')")


DataError: invalid input syntax for integer: "178.02404"
LINE 1: ...D15Z5D4F8557','Mystic World','AR0IAWL1187B9A96F9','178.02404...
                                                             ^
